## Descriptors

Короткое описание в [доке](https://docs.python.org/3/reference/datamodel.html#invoking-descriptors)

Должны быть методы `__get__`, `__set__`, `__delete__`

Обычно когда в Python мы хотим получить доступ к атрибуту класса, то сначала мы ищем его в `__dict__` инстанса класса, потом в классе, и далее выше по цепочке наследования.

In [1]:
class Car:
    def __init__(self, car_name: str, max_speed: int):
        self.car_name = car_name
        self.max_speed = max_speed

    def __str__(self):
        return f"This is {self.car_name} with max speed: {self.max_speed}"

In [2]:
car1 = Car("BMW", 250)
print(car1)

This is BMW with max speed: 250


In [3]:
car1.max_speed = 280
print(car1)

This is BMW with max speed: 280


In [5]:
car1.max_speed = "max_speed"
print(car1)

This is BMW with max speed: max_speed


In [26]:
class MaxSpeedDescriptor:
    def __init__(self):
        self._max_speed = 0

    def __get__(self, instance, owner):
        print("GET METHOD FOR MAXSPEEDDESCRIPTOR")
        return self._max_speed

    def __set__(self, instance, value):
        print("SET METHOD FOR MAXSPEEDDESCRIPTOR")
        if not isinstance(value, int):
            raise TypeError("some strange type, must be int")
        if value < 0:
            raise ValueError("max speed cannot be negative")
        self._max_speed = value

    def __delete__(self, instance):
        del self._max_speed

class Car:
    max_speed = MaxSpeedDescriptor()
    def __init__(self, car_name: str, max_speed: int):
        self.car_name = car_name
        self.max_speed = max_speed

    def __str__(self):
        return f"This is {self.car_name} with max speed: {self.max_speed}"

In [27]:
car1 = Car("BMW", 250)

SET METHOD FOR MAXSPEEDDESCRIPTOR


In [28]:
print(car1)

GET METHOD FOR MAXSPEEDDESCRIPTOR
This is BMW with max speed: 250


In [29]:
car1.max_speed = -1

SET METHOD FOR MAXSPEEDDESCRIPTOR


ValueError: ignored

In [25]:
# why do you need __get__ ?
class MaxSpeedDescriptor:
    def __init__(self):
        self._max_speed = 0

    # def __get__(self, instance, owner):
    #     print("GET METHOD FOR MAXSPEEDDESCRIPTOR")
    #     return self._max_speed

    def __set__(self, instance, value):
        print("SET METHOD FOR MAXSPEEDDESCRIPTOR")
        if not isinstance(value, int):
            raise TypeError("some strange type, must be int")
        if value < 0:
            raise ValueError("max speed cannot be negative")
        self._max_speed = value

    def __delete__(self, instance):
        del self._max_speed

class Car:
    max_speed = MaxSpeedDescriptor()
    def __init__(self, car_name: str, max_speed: int):
        self.car_name = car_name
        self.max_speed = max_speed

    def __str__(self):
        return f"This is {self.car_name} with max speed: {self.max_speed}"

car1 = Car("BMW", 250)
print(car1)

SET METHOD FOR MAXSPEEDDESCRIPTOR
This is BMW with max speed: <__main__.MaxSpeedDescriptor object at 0x7fc9e4c61930>


In [30]:
# ----

In [42]:
class MaxSpeedDescriptor:
    def __init__(self):
        self._max_speed = 0

    def __get__(self, instance, owner):
        print(type(instance))
        print(owner)
        return self._max_speed

    def __set__(self, instance, value):
        print(type(instance))
        if not isinstance(value, int):
            raise TypeError("some strange type, must be int")
        if value < 0:
            raise ValueError("max speed cannot be negative")
        self._max_speed = value

    def __delete__(self, instance):
        del self._max_speed

class Car:
    max_speed = MaxSpeedDescriptor()
    def __init__(self, car_name: str, max_speed: int):
        self.car_name = car_name
        self.max_speed = max_speed

    def __str__(self):
        return f"This is {self.car_name} with max speed: {self.max_speed}"

In [43]:
car1 = Car("BMW", 250)
print(car1)

<class '__main__.Car'>
<class '__main__.Car'>
<class '__main__.Car'>
This is BMW with max speed: 250


In [44]:
# ----

In [45]:
import os

In [53]:
class DirectorySize:
    def __get__(self, instance, owner):
        return len(os.listdir(instance.dir_name))

class Directory:
    size = DirectorySize()
    def __init__(self, dir_name):
        self.dir_name = dir_name

In [54]:
directory = Directory("./")

In [55]:
directory.size

2

In [58]:
os.listdir(".")

['.config', 'sample_data']

In [59]:
!true > temp.txt

In [60]:
os.listdir(".")

['.config', 'temp.txt', 'sample_data']

In [61]:
directory.size

3

In [63]:
# descriptor is not in the __dict__
directory.__dict__

{'dir_name': './'}

In [79]:
class Nuts:
    def __init__(self, *args, **kwargs):
        self.args = args
        self.kwargs = kwargs

    def __getattr__(self, obj):
        print("getattr is called")
        return obj

    def __delattr__(self, obj):
        print("delattr is called")
        pass

    def __getitem__(self, item):
        print("getitem is called")
        return item

    def __setitem__(self, item, value):
        print("setitem is called")
        pass

    def __delitem__(self, item):
        print("delitem is called")
        pass

    def __iter__(self):
        return self

    def __next__(self):
        raise StopIteration()

    def __str__(self):
        return "Theese are nuts"

    def __repr__(self):
        return f'Nuts({self.args}, {self.kwargs})'

In [80]:
nuts = Nuts(1,2,3,4, item1="a", item2="b")
nuts

getattr is called
getattr is called
getattr is called
getattr is called
getattr is called
getattr is called
getattr is called
getattr is called
getattr is called
getattr is called
getattr is called
getattr is called


Nuts((1, 2, 3, 4), {'item1': 'a', 'item2': 'b'})

In [81]:
print(nuts)

Theese are nuts


In [82]:
nuts.some_obj

getattr is called


'some_obj'

In [83]:
nuts.args

(1, 2, 3, 4)

In [84]:
nuts.__getattr__("args")

getattr is called


'args'

In [86]:
del nuts.zcsvegv

delattr is called


In [87]:
nuts[123]

nuts[123] = 123

del nuts[123]

getitem is called
setitem is called
delitem is called


In [88]:
class DoubleDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value * 2)

In [90]:
dd = DoubleDict()
dd["a"] = 1
dd["b"] = 2

dd

{'a': 2, 'b': 4}

In [91]:
dd.update({"c": 3})

dd

{'a': 2, 'b': 4, 'c': 3}

In [92]:
class DoubleDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value * 2)

    def update(self, other: dict):
        for k, v in other.items():
            self.__setitem__(k, v)

In [93]:
dd = DoubleDict()
dd["a"] = 1
dd["b"] = 2

dd

{'a': 2, 'b': 4}

In [94]:
dd.update({"c": 3})

dd

{'a': 2, 'b': 4, 'c': 6}

In [95]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [97]:
ad = AnswerDict()
ad["a"] = 100500

ad

{'a': 42}

In [98]:
d = dict()
d.update(ad)

In [99]:
d

{'a': 100500}

Так произошло потому, что реализация метода update не использует измененные методы. Пофиксить можно наследованием от "пользовательского словаря" из модуля collections.

In [100]:
from collections import UserDict

In [102]:
class DoubleDict(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value * 2)

In [103]:
dd = DoubleDict()
dd["a"] = 1
dd["b"] = 2

dd

{'a': 2, 'b': 4}

In [104]:
dd.update({"c": 3})

dd

{'a': 2, 'b': 4, 'c': 6}

In [105]:
class AnswerDict(UserDict):
    def __getitem__(self, key):
        return 42

In [106]:
ad = AnswerDict()
ad["a"] = 100500

ad

{'a': 100500}

In [107]:
d = dict()
d.update(ad)
d

{'a': 42}

In [114]:
class Digits:
    digits = "0123456789"
    def __getitem__(self, item):
        return self.digits[item]

    def __setitem__(self, item, value):
        # self.digits[item] = value
        pass

In [115]:
digits = Digits()

digits[0]

'0'

In [116]:
for digit in digits:
    print(digit, end=" ")
print()

0 1 2 3 4 5 6 7 8 9 


In [117]:
digits[0] = "9"

In [118]:
len(digits)

TypeError: ignored

In [119]:
class Digits:
    digits = "0123456789"
    def __getitem__(self, item):
        return self.digits[item]

    def __len__(self):
        return len(self.digits)

    def __setitem__(self, item, value):
        # self.digits[item] = value
        pass

In [120]:
digits = Digits()

len(digits)

10

In [121]:
from collections import abc

class Digits(abc.Sequence):
    digits = "0123456789"
    def __getitem__(self, item):
        return self.digits[item]

In [122]:
digits = Digits()

TypeError: ignored

In [123]:
from collections import abc

class Digits(abc.Sequence):
    digits = "0123456789"
    def __getitem__(self, item):
        return self.digits[item]
    def __len__(self):
        return len(self.digits)

In [124]:
digits = Digits()

In [125]:
[digit for digit in digits]

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [126]:
from collections import abc

class Digits(abc.Sequence):
    digits = "0123456789"
    def __getitem__(self, item):
        return self.digits[item]
    def __len__(self):
        return 2

In [127]:
digits = Digits()
[digit for digit in digits]

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [134]:
digits[:len(digits)]

'01'

In [135]:
for digit in digits:
    print(digit, end=" ")
print()

0 1 2 3 4 5 6 7 8 9 


In [142]:
from abc import ABC, abstractmethod

class MyCustomInterface(ABC):

    @abstractmethod
    def my_custom_method(self):
        raise NotImplemented()

In [143]:
class MyImpl(MyCustomInterface):
    def other_method(self):
        print("HI")

In [144]:
my_impl = MyImpl()

TypeError: ignored

In [145]:
class MyImpl(MyCustomInterface):
    def other_method(self):
        print("HI")
    def my_custom_method(self):
        print("HI!!!!")

In [146]:
my_impl = MyImpl()